<a href="https://colab.research.google.com/github/fabinprakasan/WeatheR_TypE_ClassificatioN_UsinG_CNN/blob/main/WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# Path to the weather dataset in your Colab drive
data_dir = '/content/drive/MyDrive/dataset2'

In [ ]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 1125 files belonging to 4 classes.
Using 900 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 1125 files belonging to 4 classes.
Using 225 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['cloudy', 'rainy', 'shine', 'sunrise']


In [ ]:
num_classes = len(class_names)

In [ ]:
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


29/29 [==============================] - 148s 4s/step - loss: 0.8928 - accuracy: 0.6711 - val_loss: 0.5581 - val_accuracy: 0.7778
Epoch 2/10
29/29 [==============================] - 24s 804ms/step - loss: 0.4834 - accuracy: 0.8333 - val_loss: 0.4469 - val_accuracy: 0.8267
Epoch 3/10
29/29 [==============================] - 25s 833ms/step - loss: 0.3408 - accuracy: 0.8822 - val_loss: 0.3851 - val_accuracy: 0.8311
Epoch 4/10
29/29 [==============================] - 25s 833ms/step - loss: 0.2656 - accuracy: 0.9078 - val_loss: 0.4289 - val_accuracy: 0.8444
Epoch 5/10
29/29 [==============================] - 25s 831ms/step - loss: 0.2538 - accuracy: 0.9089 - val_loss: 0.3477 - val_accuracy: 0.8844
Epoch 6/10
29/29 [==============================] - 24s 828ms/step - loss: 0.1921 - accuracy: 0.9333 - val_loss: 0.3993 - val_accuracy: 0.8533
Epoch 7/10
29/29 [==============================] - 25s 830ms/step - loss: 0.1967 - accuracy: 0.9211 - val_loss: 0.3441 - val_accuracy: 0.8800
Epoch 8/10
2

In [ ]:
def predicted_weather(img):
    img_4d = img.reshape(-1, img_height, img_width, 3)
    prediction = model.predict(img_4d)[0]
    return {class_names[i]: float(prediction[i]) for i in range(num_classes)}


In [ ]:

image = gr.inputs.Image(shape=(img_height, img_width))
label = gr.outputs.Label(num_top_classes=num_classes)

<ipython-input-17-8d46dc09661c>:1: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image = gr.inputs.Image(shape=(img_height, img_width))
<ipython-input-17-8d46dc09661c>:1: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image = gr.inputs.Image(shape=(img_height, img_width))
<ipython-input-17-8d46dc09661c>:2: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label = gr.outputs.Label(num_top_classes=num_classes)
<ipython-input-17-8d46dc09661c>:2: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label = gr.outputs.Label(num_top_classes=num_classes)


In [22]:
gr.Interface(fn=predicted_weather, inputs=image, outputs=label, interpretation='default').launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 22ms/step
Keyboard interruption in main thread... closing server.
